import libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import tensorflow as tf

from keras.utils import normalize
from tensorflow.keras.layers import Flatten,Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

import dataset from kaggel

In [3]:
!pip install kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d shivanandmn/multilabel-classification-dataset

from zipfile import ZipFile
dataset='/content/multilabel-classification-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('data extracted')

  0% 0.00/11.4M [00:00<?, ?B/s]
100% 11.4M/11.4M [00:00<00:00, 245MB/s]
data extracted


data preparation

In [4]:
data_train=pd.read_csv('/content/train.csv')
data_test=pd.read_csv('/content/test.csv')

In [5]:
data_horizontal = data_train.columns
print(data_horizontal)
x_train = data_train[data_horizontal[1:3]]
y_train = data_train[data_horizontal[3:]]
x_test = data_test[data_horizontal[1:3]]

Index(['ID', 'TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'],
      dtype='object')


In [6]:
data_train.shape

(20972, 9)

In [7]:
data_test.shape

(8989, 3)

In [8]:
data_train.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [9]:
data_test.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


fun to convert text to vector

In [10]:

def text_to_vectors(df, max_len=128, max_features=None, vocab_size=None):

  text_data = df['TITLE'] + ' ' + df['ABSTRACT']

  tokenizer = Tokenizer(num_words=max_features if max_features else vocab_size)
  tokenizer.fit_on_texts(text_data)
  sequences = tokenizer.texts_to_sequences(text_data)

  padded_sequences = pad_sequences(sequences, maxlen=max_len)

  return padded_sequences

In [11]:
x_train = text_to_vectors(x_train)
x_test = text_to_vectors(x_test)

In [12]:
print('x_train',x_train)

x_train [[  230    85    12 ... 13191     9    65]
 [    0     0     0 ...    28   364  2344]
 [    0     0     0 ...     1   990  2669]
 ...
 [    5    17  1118 ...     1  3454  2424]
 [    2   180   298 ...     6   206   847]
 [   10  8115  3446 ...     4  1498  2077]]


In [13]:
print('x_test',x_test)

x_test [[    0     0     0 ...    11  3622   113]
 [  544     2   858 ...     2     1 15277]
 [11767  1484  2512 ...  2512  1093   194]
 ...
 [    0     0     0 ...     2  2531  2938]
 [    4  1533   211 ...   630  1847  3268]
 [    0     0     0 ...     1   121  3835]]


build our model

In [14]:

input_shape=x_train.shape[1:]
model= tf.keras.Sequential()
model.add(Dense(64, activation="relu", input_shape=input_shape)),
model.add(Dense(32, activation="relu")),
model.add(Dense(6, activation="sigmoid"))

compile our model

In [15]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


fit our model and know accuracy

In [16]:

model.fit(x_train, y_train, epochs=30, batch_size=32)

Epoch 1/30
656/656 [==============================] - 3s 2ms/step - loss: 104.1247 - accuracy: 0.3463
Epoch 2/30
656/656 [==============================] - 2s 3ms/step - loss: 0.7238 - accuracy: 0.4094
Epoch 3/30
656/656 [==============================] - 2s 2ms/step - loss: 0.5194 - accuracy: 0.4103
Epoch 4/30
656/656 [==============================] - 1s 2ms/step - loss: 0.4726 - accuracy: 0.4105
Epoch 5/30
656/656 [==============================] - 1s 2ms/step - loss: 0.4523 - accuracy: 0.4104
Epoch 6/30
656/656 [==============================] - 1s 2ms/step - loss: 0.4449 - accuracy: 0.4103
Epoch 7/30
656/656 [==============================] - 1s 2ms/step - loss: 0.4414 - accuracy: 0.4103
Epoch 8/30
656/656 [==============================] - 1s 2ms/step - loss: 0.4386 - accuracy: 0.4103
Epoch 9/30
656/656 [==============================] - 1s 2ms/step - loss: 0.4367 - accuracy: 0.4103
Epoch 10/30
656/656 [==============================] - 1s 2ms/step - loss: 0.4357 - accuracy: 0.41

In [18]:
score = model.evaluate(x_train,y_train)
print(' accuracy:', score[1])

656/656 [==============================] - 1s 1ms/step - loss: 0.4349 - accuracy: 0.4099
 accuracy: 0.4098798334598541
